In [3]:
!conda uninstall --yes networkx
!pip install --upgrade git+git://github.com/networkx/networkx.git#egg=networkx

Fetching package metadata .........
Solving package specifications: ..........


PackageNotFoundError: Package not found: '' no packages found to remove from environment: /opt/conda


  Cloning git://github.com/networkx/networkx.git to /tmp/pip-build-p62we251/networkx
  Found existing installation: decorator 4.0.10
    Uninstalling decorator-4.0.10:
      Successfully uninstalled decorator-4.0.10
  Running setup.py install for networkx ... - \ | / - \ | / - \ | / done


In [8]:
import networkx as nx
import graph_tool as gt